In [10]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [11]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to C:\Users\Mukteshwar
[nltk_data]     Nath\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [14]:
# --- Make sure required NLTK data is available ---
for resource in ("punkt", "wordnet", "omw-1.4"):
    try:
        nltk.data.find(f"tokenizers/{resource}")  # quick existence check for punkt
    except LookupError:
        nltk.download(resource, quiet=True)

lemmatizer = WordNetLemmatizer()

ignore_words = {'?', '!'}

# --- Load intents file ---
with open('intents.json', 'r', encoding='utf-8') as f:
    intents = json.load(f)

words = []
classes = []
documents = []  # (token_list, tag)

# --- Build raw word list & documents ---
for intent in intents.get('intents', []):
    tag = intent['tag']
    if tag not in classes:
        classes.append(tag)

    for pattern in intent.get('patterns', []):
        tokens = nltk.word_tokenize(pattern)
        documents.append((tokens, tag))
        words.extend(tokens)

# --- Build cleaned vocabulary ---
vocab = []
for w in words:
    if w in ignore_words:
        continue
    lw = w.lower()
    if not lw.isalpha():
        continue
    vocab.append(lemmatizer.lemmatize(lw))

words = sorted(set(vocab))

# --- Finalize classes ---
classes = sorted(set(classes))

print(f"{len(documents)} documents")
print(f"{len(classes)} classes: {classes}")
print(f"{len(words)} unique lemmatized words: {words}")

# --- Persist vocabulary & labels ---
with open('words.pkl', 'wb') as f:
    pickle.dump(words, f)
with open('classes.pkl', 'wb') as f:
    pickle.dump(classes, f)

# --- Create training data (bag-of-words + one-hot intent) ---
training = []
output_empty = [0] * len(classes)

for token_list, tag in documents:
    # normalize pattern tokens same way vocab was built
    cleaned = []
    for w in token_list:
        if w in ignore_words:
            continue
        lw = w.lower()
        if not lw.isalpha():
            continue
        cleaned.append(lemmatizer.lemmatize(lw))

    cleaned_set = set(cleaned)

    # bag vector
    bag = [1 if w in cleaned_set else 0 for w in words]

    # one-hot intent
    output_row = output_empty[:]
    output_row[classes.index(tag)] = 1

    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

print("Training data created.")


47 documents
14 classes: ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'noanswer', 'options', 'pharmacy_search', 'search_blood_pressure_by_patient_id', 'search_hospital_by_params', 'search_hospital_by_type', 'search_pharmacy_by_name', 'thanks']
86 unique lemmatized words: ['a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that',

In [13]:
import tensorflow as tf
import random, os
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# --- Convert training data from lists to float32 arrays ---
X = np.array(train_x, dtype="float32")  # shape: (num_samples, vocab_size)
y = np.array(train_y, dtype="float32")  # shape: (num_samples, num_classes)

# --- Build model ---
model = Sequential([
    Dense(128, input_shape=(X.shape[1],), activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(y.shape[1], activation='softmax')
])

opt = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)

# --- Compile ---
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# --- Train ---
history = model.fit(
    X, y,
    epochs=200,
    batch_size=5,
    verbose=1
)

Epoch 1/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.0174 - loss: 2.6711 
Epoch 2/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1616 - loss: 2.5453   
Epoch 3/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2110 - loss: 2.4411  
Epoch 4/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1544 - loss: 2.3365  
Epoch 5/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3123 - loss: 2.1334     
Epoch 6/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4281 - loss: 1.9120 
Epoch 7/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3456 - loss: 1.8146 
Epoch 8/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5463 - loss: 1.6301  
Epoch 9/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4476 - loss: 1.5618 
Epoch 10/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5407 - loss: 1.4622
Epoch 11/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.5427 - loss: 1.3466
Epoch 12/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 

In [9]:
# --- Save model ---
model.save('chatbot_model.h5')  # don't pass history here

# --- Save training history separately (optional) ---
with open('chatbot_history.pkl', 'wb') as f:
    pickle.dump(history.history, f)

print("Model created and saved.")

Model created and saved.
